### Intro
* A replication for Rivkin's paper [Balancing Search and Stability: Interdependencies Among Elements of Organizational Design](https://pubsonline.informs.org/doi/abs/10.1287/mnsc.49.3.290.12740?journalCode=mnsc) on Management Science in 2003. 
* Organization elements:
    1. Vertical hierarchy, a CEO sit about a set of departments
    2. Incentive system, managers may be rewarded on the basis of overall firm performance or on the performance of individual departments
    3. Decomposition, the nature of assigning decision rights in a way that places related decisions under a single manager
* Contextual variables:
    1. The underlying pattern of interaction among a firm's decisions
    2. Limits on the ability of managers
* This study focuses on the interdependencies among theses five elements in a firm

### NK model  
###### Representation
* Vertical hierarchy is classified into two categories:  
    1. rubberstamping CEO, the decision makeing is totally decentralized
    2. active CEO, the CEO requires P sub decisions from manager A and P another sub decisions from manager B, CEO selects from P^2 combination. CEO has limit ability, it is represented by the cardinality of configuration examed by CEO, $$AltCEO$$ 
* Incentive system is represented by the knowledge to configuration table, that is, a manager motivated by the whole firm perfomance would take all decisions' contribution into consideration when choose from alternatives. Manager's evaluation of configuration could be notated as 
$$
P^{'}(d) = \{[C_2(d),C_3(d),C_4(d),C_5(d)] + Incent*[C_1(d),C_6(d)]\}/6,\\ where\ the\ manager\ would\ not\ take\ the\ contribution\ of\ c_1\ and\ c_6\ when\ incent==0
$$
* Decomposition is represented by the allocation of decisions,e.g.,decision 2-5 belong to manager B while decision 1 and decision 6 belong to manager A in a firm with total 6 decisions, so the decision allocation is abbbba.
* The underlying pattern of interaction among a firm's decisions is represented by the level of K and the pattern of influence matrix in NK model
* Manager ability is represented by the cardinality of alternatives configuration considered by managers, 
$$ AltSub $$
* Information flow is represented by the cardinality of proposals come to CEO in the settings of vertical hierarchy

### Simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
def createInfluenceMatrix(N,K,K_within=None,K_between=None):
    IM = np.eye(N)
    if not K_within:
        for i in range(N):
            probs = [1/(N-1)]*i + [0] +[1/(N-1)]*(N-1-i)
            ids = np.random.choice(N,K,p=probs,replace=False)
            for index in ids:
                IM[i][index] = 1
    else:
        for i in range(N):
            if i//(N//2)<1:
                within = [j for j in range(N//2)]
                between = [j for j in range(N//2,N)]
                probs = [1/(N//2-1)]*i+[0]+[1/(N//2-1)]*(N//2-1-i)
                ids_within = np.random.choice(within,K_within,p=probs,replace=False)
                ids_between = np.random.choice(between,K_between,replace=False)
                for index in ids_within:
                    IM[i][index] = 1
                for index in ids_between:
                    IM[i][index] = 1
                    
            else:
                within = [j for j in range(N//2,N)]
                between = [j for j in range(N//2)]
                probs = [1/(N//2-1)]*(i-N//2)+[0]+[1/(N//2-1)]*(N-1-i)
                ids_between = np.random.choice(between,K_between,replace=False)
                ids_within = np.random.choice(within,K_within,p=probs,replace=False)
                for index in ids_within:
                    IM[i][index] = 1
                for index in ids_between:
                    IM[i][index] = 1
                    
    IM_dic = defaultdict(list)
    for i in range(len(IM)):
        for j in range(len(IM[0])):
            if i==j or IM[i][j]==0:
                continue
            else:
                IM_dic[i].append(j)
    return IM,IM_dic

In [3]:
def createFitnessConfig(IM):
    FC = defaultdict(dict)
    for row in range(len(IM)):
        k = int(sum(IM[row]))
        for i in range(pow(2,k)):
            FC[row][i] = np.random.uniform(0,1)
    return FC

In [152]:
def calculate_Fitness(state,IM_dic,FitnessConfig,idx=None):
    if not idx:
        res = 0.0
        for i in range(len(state)):
            dependency = IM_dic[i]
            bin_index = "".join([str(state[j]) for j in dependency])
            if state[i]==0:
                bin_index = "0" + bin_index
            else:
                bin_index = "1" + bin_index
            index = int(bin_index,2)
            res+=FitnessConfig[i][index]
        return res/len(state)
    else:
        res = 0.0
        for i in range(len(idx)):
            dependency = IM_dic[i]
            bin_index = "".join([str(state[j]) for j in dependency])
            if state[i]==0:
                bin_index = "0" + bin_index
            else:
                bin_index = "1" + bin_index
            index = int(bin_index,2)
            res+=FitnessConfig[i][index]
        return res/len(idx)

In [153]:
def initializeState(N):
    return np.random.choice([0,1],N)

In [276]:
def Subfunction(state,AltSub,Incent,IM_dic,FG,idx,P):
    pool_A = []
    
    if Incent==1:
        temp_fitness_value = calculate_Fitness(state,IM_dic,FG)
    else:
        temp_fitness_value = calculate_Fitness(state,IM_dic,FG,idx)
    pool_A.append((list(state),float(temp_fitness_value)))
    
    

    if AltSub==1:
        order = np.random.choice(idx,1,replace=False)
        AltSub_order = [order]
    elif AltSub==4:
        order = np.random.choice(idx,2,replace=False)
        AltSub_order = [[i] for i in idx]+[order]
    elif AltSub==7:
        AltSub_order = [[i] for i in idx] + [[idx[0],idx[1]],[idx[0],idx[2]],[idx[1],idx[2]]]+ [idx]
    else:
        print("False!!!!!!!!")
    
    for i in range(AltSub):
        ids = AltSub_order[i]
        temp_state = list(state)
        for index in ids:
            temp_state[index] = temp_state[index]^1

            if Incent==1:
                temp_fitness_value = calculate_Fitness(temp_state,IM_dic,FG)
            else:
                temp_fitness_value = calculate_Fitness(temp_state,IM_dic,FG,idx)
            pool_A.append((list(temp_state),float(temp_fitness_value)))
    pool_A = sorted(pool_A,key=lambda x:-x[-1])
    return pool_A[:P]

In [177]:
def reconstruct(A_idx,B_idx,A,B):
    l = len(A)
    res = [0 for i in range(l)]
    
    for i in range(len(A_idx)):
        res[A_idx[i]] = A[A_idx[i]]
    for i in range(len(B_idx)):
        res[B_idx[i]] = B[B_idx[i]]
    return list(res)

In [227]:
def firmAdap(IM,IM_dic,FG,initialState,P,AltCEO,AltSub,allocation,Incent,iteration):
    A_idx = []
    B_idx = []
    for i,a in enumerate(allocation):
        if a=="a":
            A_idx.append(i)
        else:
            B_idx.append(i)
    
    state = list(initialState)
    fitness_value = calculate_Fitness(state,IM_dic,FG)
    res = [fitness_value]
    
    for step in range(iteration):
        # for manager A
        pool_A = Subfunction(state,AltSub,Incent,IM_dic,FG,A_idx,P)
        # for manager B
        pool_B = Subfunction(state,AltSub,Incent,IM_dic,FG,B_idx,P)
        # for CEO
        if AltCEO==0:
            state = reconstruct(A_idx,B_idx,pool_A[0][0],pool_B[0][0])
            fitness_value = calculate_Fitness(state,IM_dic,FG)
            res.append(fitness_value)
        else:
            pool_A.append((state,calculate_Fitness(state,IM_dic,FG)))
            pool_B.append((state,calculate_Fitness(state,IM_dic,FG)))
            idx = np.random.choice(len(pool_A)*len(pool_B),AltCEO,replace=False)
            pool_CEO = []
            for index in idx:
                A_index = index//len(pool_A)
                B_index = index%len(pool_B)
#                 print(A_index,B_index)
                temp_state = reconstruct(A_idx,B_idx,pool_A[A_index][0],pool_B[B_index][0])
                temp_fitness_value = calculate_Fitness(temp_state,IM_dic,FG)
                pool_CEO.append((list(temp_state),float(temp_fitness_value)))
            pool_CEO = sorted(pool_CEO,key=lambda x:-x[-1])
            temp_state,temp_fitness_value = pool_CEO[0]
            if temp_fitness_value>fitness_value:
                fitness_value = temp_fitness_value
                state = temp_state
            res.append(fitness_value)
    return np.array(res)

###### Active vs. Passive Vertical Hierarchy and Degree of Interaction  
In this experiment, we fix the decision allocation to aaabbb, AltSub=1 and exam the interaction among Information Flow(P), CEO ability(rubberstamping, low ability, high ability) and the interdependencies among decision (K)

In [269]:
def Simulation(N,K,landscape_number,K_between=None,K_within=None,epoch=100,Incent=0,AltSub=1):
    """
    four firm differ in terms of information flow (P) and CEO ability (AltCEO) 
    """
    IM,IM_dic = createInfluenceMatrix(N,K,K_within,K_between)
    resA = []
    resB = []
    resC = []
    resD = []
    for landscape in range(landscape_number):
        FG = createFitnessConfig(IM)
        state = initializeState(N)
        
        # firmA
        stateA = list(state)
        resA.append(firmAdap(IM,IM_dic,FG,stateA,1,0,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmB
        stateB = list(state)
        resB.append(firmAdap(IM,IM_dic,FG,stateB,1,1,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmC
        stateC = list(state)
        resC.append(firmAdap(IM,IM_dic,FG,stateC,2,1,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmD
        stateD = list(state)
        resD.append(firmAdap(IM,IM_dic,FG,stateD,1,3,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmD
    resA = np.array(resA)
    resB = np.array(resB)
    resC = np.array(resC)
    resD = np.array(resD)
    return resA,resB,resC,resD

In [266]:
# in the short run -> 4 epoch
for k in range(6):
    A,B,C,D = Simulation(6,k,1000,epoch=4)
    print(np.mean(A),np.mean(B),np.mean(C),np.mean(D))

0.564555410232719 0.5402620668826039 0.555145384534358 0.5639430606331804
0.5922490910695429 0.5721363242809336 0.5645429490072573 0.6087869339059325
0.5995777647397664 0.584810463608077 0.5804276050047188 0.6218993072782338
0.6086543966154936 0.6047304758611172 0.5924246639907019 0.6448291338413356
0.6083162445820917 0.6064078686259763 0.5991705719767314 0.6401030470722792
0.5945163014009381 0.5984901103069753 0.5933963029920176 0.6312826937712376


In [263]:
# in the long run -> 100 epoch
for k in range(6):
    A,B,C,D = Simulation(6,k,1000)
    print(np.mean(A),np.mean(B),np.mean(C),np.mean(D))

0.583622531977713 0.583622531977713 0.6666927049549156 0.583622531977713
0.5921993383257563 0.5936423757928062 0.6950516540264088 0.5947553998909124
0.6257090742230609 0.6470961436353054 0.7143687618185744 0.6467349331120559
0.6407587287886225 0.6646296464662964 0.7143039130232087 0.6680590559633359
0.6330990025996882 0.6572090847766128 0.7136334358473088 0.6587648646359425
0.6313867738385586 0.6531332396064442 0.7097576937563476 0.658157699840837


* The main difference between original paper and my replication is the effect of complexity, the original paper reveals a monotonically decrease relation while my replication gets a reversed-U shape
* However, the main findings remain same,
    1. In the short run, active hierarchy could be harmful but as the complexity/interdependency among decision increases, active hierarchy would be benificial 
    2. On the long run, we could see active hierarchy would outperform much more than rubberstamping CEO when there is a rich flow of information

###### Active Vertical Hierarchy and Managerial Ability  
In this experiment, we relax the constraint on AltSub, that is, sub managers of department could have different ability. 

In [278]:
for AltSub in [4,7]:
    print("----------AltSub==%d--------"%AltSub)
    for k in range(6):
        A,B,C,D = Simulation(6,k,1000,AltSub=AltSub)
        print(np.mean(A),np.mean(B),np.mean(C),np.mean(D))

----------AltSub==4--------
0.5877005180372205 0.5877005180372205 0.6150228238944683 0.5877005180372205
0.6157381326919933 0.644227644452515 0.6835193272102327 0.6497474444538335
0.6368852716765556 0.6750914764979685 0.7035757918751974 0.6795260801959437
0.6297522595305385 0.6748665360767476 0.714818996067321 0.6837655168674307
0.6264609668104015 0.6768914426275792 0.7195615867008834 0.6834282623641972
0.602769662898789 0.6730077350068222 0.7138266392590098 0.6801223410322558
----------AltSub==7--------
0.5827555195371483 0.5827555195371483 0.6128849000191383 0.5827555195371483
0.595067213514046 0.6263043195493849 0.659698390575521 0.6305358793353893
0.5984239109847372 0.6453251310649296 0.6826074024868326 0.6514674764749929
0.570826952668328 0.6689647247161927 0.7100935452594171 0.6774324048105864
0.5529994132154524 0.6756249222543893 0.709405138231572 0.685661502485113
0.536277028227728 0.665906953701869 0.7028634507941901 0.6759997720221008


* Same difference between monotonically decrease function and reversed-U shape still exist
* These findings still hold:
    1. Hiring smarter managers could undermine firms' performance, especially when decisions richly interact and CEO tubber-stamps decisions  
    2. Hiring smarter managers and active hierarchy could be complementary if we regard CEO as a mechanism to integrate
    3. Smarter managers could prevent firm to search because it hide search strategy behind its prefered decisions

###### Active Vertical Hierarchy and Firm-Wide Incentives  
In this experiment, we relax the constraint on Incent, that is, managers would work in order to enhance the performance of the whole firm

In [279]:
for AltSub in [1,4,7]:
    print("----------AltSub==%d--------"%AltSub)
    for k in range(6):
        A,B,C,D = Simulation(6,k,1000,AltSub=AltSub,Incent=1)
        print(np.mean(A),np.mean(B),np.mean(C),np.mean(D))

----------AltSub==1--------
0.6703710786932211 0.6703710786932211 0.6703710786932211 0.6703710786932211
0.6924704511423699 0.6924050029555262 0.6989286061353064 0.6918674031618045
0.7087420135819467 0.7061728882401369 0.719090268837966 0.7057648879566574
0.6921272940713266 0.69137199148384 0.706917546022417 0.6922811311804358
0.6909080583243103 0.6887005304135266 0.7146543656269102 0.6878218714849216
0.6811128481131674 0.6785143046179883 0.708966144805103 0.6818314681461273
----------AltSub==4--------
0.6642959805298165 0.6642959805298165 0.6642959805298165 0.6642959805298165
0.6824726911659277 0.6914617162150333 0.694776625205427 0.6932322310012267
0.6895829345327388 0.7064209551381575 0.7154011474672466 0.7123697503519925
0.7043427531735394 0.7230276753696175 0.7301046667182106 0.7232546309371105
0.6930356581442437 0.7197871782295333 0.7263815514530374 0.7213418325366207
0.6495511016615609 0.711192725411375 0.7198895336396398 0.7124651466314289
----------AltSub==7--------
0.667034347

* Firm-wide incentives increase firms' overall performance
* The following conclusion holds for Altsub=7:
    1. Firm-wide incentives and active hierarchy could be complementary, the increasement from active hierarchy is significant greater when Incent equals to 1 compared with Incent equals to 0.

###### Active Vertical Hierarchy and Decision Decomposition  
In this experiment, we relax the assumption about the pattern of complexity (Influence Matrix) and allow it to be diagonal. The allocation,e.g., aaabbb could also be changed to aabbba

In [280]:
def Simulation(N,K,landscape_number,K_between=None,K_within=None,epoch=100,Incent=0,AltSub=1):
    """
    four firm differ in terms of information flow (P) and CEO ability (AltCEO) 
    """
    IM,IM_dic = createInfluenceMatrix(N,K,K_within,K_between)
    resA = []
    resB = []
    resC = []
    resD = []
    for landscape in range(landscape_number):
        FG = createFitnessConfig(IM)
        state = initializeState(N)
        
        # firmA
        stateA = list(state)
        resA.append(firmAdap(IM,IM_dic,FG,stateA,1,0,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmB
        stateB = list(state)
        resB.append(firmAdap(IM,IM_dic,FG,stateB,2,3,AltSub,"aaabbb",Incent,epoch)[-1])
        
        # firmC
        stateC = list(state)
        resC.append(firmAdap(IM,IM_dic,FG,stateC,1,0,AltSub,"aabbba",Incent,epoch)[-1])
        
        # firmD
        stateD = list(state)
        resD.append(firmAdap(IM,IM_dic,FG,stateD,2,3,AltSub,"aabbba",Incent,epoch)[-1])
        
        # firmD
    resA = np.array(resA)
    resB = np.array(resB)
    resC = np.array(resC)
    resD = np.array(resD)
    return resA,resB,resC,resD

In [283]:
A,B,C,D = Simulation(6,2,1000,K_between=0,K_within=2)
print(np.mean(A),np.mean(B),np.mean(C),np.mean(D))

0.6012669158753198 0.6926390404669579 0.6014213974859122 0.7240606037093439


* D-C > B-A indicates that active hierarchy is more benifical when the task could not be completel decomposed 
* The best performance comes from D where there is an active hierarchy and unnecessay overlap, it challenges the conventional wisdom that firms should strive for complete decomposition. 

### Potential extension  
* competition among firms -> once a decision is made, the probability of other firms making same decision in the same industry should be decreased -> cannot stand on local peak together. The decision of search and stablization of other firms is visiable in online context (e.g.,APPLE STORE).
* this paper assume equal ability of subs, however the reality is subs with more ability get promotion and make more decisions/ earn more freedom
* short-run v.s. long-run performance indicates the importance of speed in analysis. 
* what about extending the decision space -> new area